# [실습 11] RAG 전 과정: 검색 증강 생성 실습

이 노트북은 **RAG (Retrieval-Augmented Generation)**의 전체 과정을 단계별로 실습하는 코드입니다.
**[코드 옆 주석]**과 **[문법 설명]**을 함께 읽어보세요.

## 1. 라이브러리 임포트 (Import)
**필요한 도구들을 공구상자에서 꺼내는 단계**입니다.

### 🦁 사자개 문법 교실: `from ... import ...`
*   `import os`: `os`라는 도구상자를 **통째로** 가져옵니다.
*   `from A import B`: `A`라는 큰 상자에서 **`B`라는 특정 도구만 쏙** 꺼내옵니다. (메모리 절약)

In [ ]:
import os
from dotenv import load_dotenv

# 1. LLM 모델과 임베딩(번역기) 도구 임포트
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 2. 텍스트를 자르는 도구 (Splitter)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 3. 벡터 데이터베이스 (Vector DB)
from langchain_chroma import Chroma

# 4. 문서를 감싸는 객체 (Document)
from langchain_core.documents import Document

## 2. 환경 설정 (Setup)
**API 키를 금고(.env)에서 꺼내오는 단계**입니다.

### 🦁 사자개 문법 교실: `f-string` (문자열 포맷팅)
*   `f"... {변수} ..."`: 문자열 앞에 `f`를 붙이면, 중괄호 `{}` 안에 **변수**를 넣을 수 있습니다.
*   `api_key[:5]`: **슬라이싱(Slicing)**. 문자열의 **처음부터 5글자**만 자른다는 뜻입니다.

In [ ]:
# .env 파일 로드
load_dotenv()

# 환경변수에서 API Key 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# 키가 잘 로드되었는지 확인 (보안을 위해 일부만 출력)
if api_key:
    print(f"API Key Loaded: {api_key[:5]}...")
else:
    print("API Key가 없습니다. .env 파일을 확인해주세요.")

## 3. 데이터 준비 (Load)
**AI에게 가르칠 지식(컨닝 페이퍼)을 준비하는 단계**입니다.

### 🦁 사자개 문법 교실: `"""..."""` (따옴표 3개)
*   일반 따옴표(`"`)는 한 줄짜리 글을 쓸 때 씁니다.
*   따옴표 3개(`"""`)를 쓰면 **여러 줄(Multi-line)**에 걸친 긴 글을 변수에 담을 수 있습니다.

In [ ]:
# 가상의 회사 사규 데이터
raw_text = """
[주식회사 사자개 사규]
제 1조 (목적) 본 규정은 사자개 주식회사의 복지를 규정한다.
제 2조 (근무시간) 근무시간은 오전 10시부터 오후 5시까지로 한다. (주 35시간)
제 3조 (복지 포인트) 전 직원은 매년 1월 1일, 복지 포인트 300만 원을 지급받는다.
제 4조 (휴가) 연차는 무제한으로 사용할 수 있다. 다만, 2주 이상 연속 사용 시 팀장 승인이 필요하다.
제 5조 (간식) 탕비실에는 항상 몬스터 에너지 드링크와 마카롱을 구비해야 한다.
"""

print("데이터 준비 완료.")

## 4. 텍스트 분할 (Split)
**문서를 AI가 이해하기 좋은 크기로 자르는 단계(Chunking)**입니다.

### 🦁 사자개 문법 교실: `Class(인자=값)`
*   `RecursiveCharacterTextSplitter(...)`: **붕어빵 틀(Class)**을 사용해서 **붕어빵(Instance)**을 찍어내는 과정입니다.
*   `chunk_size=100`: **키워드 인자(Keyword Argument)**. 순서를 몰라도 이름(`chunk_size`)으로 값을 전달할 수 있습니다.

In [ ]:
# 1. 자르는 규칙 설정 (칼 준비)
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,    # 한 조각의 크기
    chunk_overlap = 20   # 겹치는 구간 (직소퍼즐의 연결부위)
)

# 2. 자르기 수행 (String -> Document 객체로 변환)
# create_documents([...]) : 리스트 형태로 데이터를 넣어줘야 합니다.
chunks = splitter.create_documents([raw_text])

print(f"총 {len(chunks)}개의 조각(Chunk)이 생성되었습니다.")
print(f"첫 번째 조각 미리보기: {chunks[0].page_content}")

## 5. 임베딩 및 저장 (Embed & Store)
**가장 중요한 마법의 단계**입니다.

### 🦁 사자개 문법 교실: `Chroma.from_documents`
*   `from_documents`: 이건 **클래스 메서드(Class Method)**라고 합니다.
*   보통은 `db = Chroma()` 이렇게 빈 DB를 만들고 넣는데,
*   `from_documents`는 **"문서를 주면 알아서 DB를 만들어서 뱉어라"** 하는 편리한 기능(Factory)입니다.

In [ ]:
# Chroma DB를 메모리에 생성하고 문서를 저장함
vector_db = Chroma.from_documents(
    documents = chunks,                 # 쪼개진 문서 조각들
    embedding = OpenAIEmbeddings()      # 사용할 번역기 (글자 -> 숫자)
)

print("데이터베이스 저장 완료. (이제 검색 가능!)")

## 6. 질문 및 검색 (Retrieve)
이제 도서관(DB)에 질문을 던져서 **관련된 문서(컨닝 페이퍼)**를 찾아옵니다.

### 🦁 사자개 문법 교실: `documents[0].page_content`
*   `retrieved_docs`: 검색된 문서들의 **리스트(List)**입니다. (`[Doc1, Doc2, ...] `)
*   `[0]`: 리스트의 **첫 번째(0번)** 녀석을 가져옵니다.
*   `.page_content`: 그 문서 객체 안에 있는 **변수(속성)**를 꺼냅니다.

In [ ]:
query = "회사 복지 포인트 얼마나 줘?"

# 유사도 검색 수행
retrieved_docs = vector_db.similarity_search(query, k=2)

print(f"질문: {query}")
print("\n=== [검색된 컨닝 페이퍼] ===")
print(retrieved_docs[0].page_content) # 가장 유사도가 높은 문서 내용

## 7. 답변 생성 (Generate)
**RAG의 완성**입니다.
찾아온 문서(`retrieved_docs`)를 프롬프트에 끼워 넣어서 AI에게 대답하게 시킵니다.

In [ ]:
# 1. 답변할 모델 설정 (temperature=0: 창의성 끄고 팩트만 전달)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. 프롬프트 조립 (Context Injection)
# 검색된 내용을 [참고 문서]에 넣어줍니다.
prompt = f"""
당신은 인사팀 AI 챗봇입니다.
아래의 [참고 문서]를 바탕으로 질문에 답하세요.

[참고 문서]
{retrieved_docs[0].page_content}

질문: {query}
"""

# 3. AI에게 답변 생성 요청
response = llm.invoke(prompt)

print("\n=== [AI 최종 답변] ===")
print(response.content)